# WORK IN PROGRESS

Not sure what widget options would be best and how to write a performant callback that selects and processes values for each timestep

In [1]:
import pyvista as pv
import numpy as np
import xarray as xr
from JulesD3D.utils import quickDF
from JulesD3D.processNetCDF import fixMeshGrid, addVectorSum, addDepth, makeVelocity, addUnderlayerCoords
from JulesD3D.plotPyVista import makeBottomSurface, makeStructuredGridDepth, easyTimeFormat
from cmocean.cm import deep
import matplotlib.pyplot as plt
from JulesD3D.utils import ncFilepicker
from os import path

In [2]:
folder = '/Users/julesblom/ThesisResults/'

nc_filename = ncFilepicker(folder)

In [3]:
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom/The…

In [4]:
basename = path.basename(nc_filename.value)
plot_title, _ = path.splitext(basename)
plot_title

'trim-36km_200m_W60ChannelRun01MoreOutput'

In [5]:
trim = xr.open_dataset(nc_filename.value) # Open with xarray : DataSet : A collection of DataArrays

In [6]:
if 'velocity' not in trim:
    trim = fixMeshGrid(trim, trim.XZ.values, trim.YZ.values, mystery_flag=True)
    trim = addDepth(trim)
    trim = makeVelocity(trim)
    trim = addUnderlayerCoords(trim)
else:
    print("DataSet is preprocessed, 'velocity' is already in DataSet")

------ Fixing mesh grid, assuming a uniform grid ------
x_gridstep:	 200.0 m
y_gridstep:	 200.0 m
width:		 26000.0 m
length:		 36200.0 m
132 x 182 grid


### Process data vars

In [7]:
concentration = trim.R1.transpose('time', 'LSTSCI', 'M', 'N', 'KMAXOUT_RESTR')
# concentration_sand = concentration.isel(LSTSCI=0)
concentration_silt = concentration.isel(LSTSCI=1)
# concentration_silt_at_time = concentration_silt.isel(time=5)
# concentration_silt_at_time = concentration_silt_at_time.assign_attrs({'long_name': 'Silt concentrations percentage per layer in zeta point',
#                                                               'units': '%',
#                                                               'grid': 'grid',
#                                                               'location': 'face'})

In [8]:
concentration_silt

<xarray.DataArray 'R1' (time: 191, M: 132, N: 182, KMAXOUT_RESTR: 80)>
[367086720 values with dtype=float32]
Coordinates:
    XZ             (M, N) int64 0 0 0 0 0 0 ... 26100 26100 26100 26100 26100
    YZ             (M, N) int64 0 200 400 600 800 ... 35600 35800 36000 36200
  * KMAXOUT_RESTR  (KMAXOUT_RESTR) int32 0 1 2 3 4 5 6 ... 73 74 75 76 77 78 79
  * time           (time) datetime64[ns] 2020-02-01 ... 2020-02-01T19:00:00
Dimensions without coordinates: M, N
Attributes:
    long_name:  Concentrations per layer in zeta point
    units:      1
    grid:       grid
    location:   face

In [9]:
# Reorder velocity dims to match x, y, z
velocity = trim.velocity.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
velocity.dims

('time', 'M', 'N', 'KMAXOUT_RESTR')

In [10]:
density = trim.RHO.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
density.dims

('time', 'M', 'N', 'KMAXOUT_RESTR')

### Turn Delft3D output into PyVista grid

In [11]:
bottom_surface = makeBottomSurface(trim, timestep=0)
bottom_surface

StructuredGrid (0x1242fe8a0)
  N Cells:	23400
  N Points:	23711
  X Bounds:	0.000e+00, 2.600e+04
  Y Bounds:	1.000e+02, 3.610e+04
  Z Bounds:	-6.999e+02, -0.000e+00
  Dimensions:	131, 181, 1
  N Arrays:	1

In [12]:
depth_centers_grid = makeStructuredGridDepth(trim, keyword='SIG_LYR')

'depth' DataArray already found in DataSet!
xyz_layers.shape (1921920, 3)


In [13]:
plotting_options = dict(lighting=True, ambient=0.9, specular=0.1, specular_power=1) #show_edges=False ,

sargs = dict(
    title_font_size=30,
    label_font_size=28,
    shadow=True,
    n_labels=6,
    fmt="%.1f",
    font_family="arial",
)

outline = depth_centers_grid.outline()

# Interactive Density

In [14]:
depth_centers_grid['Density'] = density.isel(time=initial_timestep).values.ravel()
# density_threshold_lim = [1025.25, 1040]
# threshed_density_centers = depth_centers_grid.threshold(density_threshold_lim, scalars="Density")

density_clim = (1020, 1040)
opacity_trick = [0] * 6 + [0.9] * 15 # should 'map' to steps of clim 

NameError: name 'initial_timestep' is not defined

In [ ]:
def densityCallback(val):
    p.remove_actor(initTimeActor)
    outputstep = int(val)
    Density = density.isel(time=outputstep).values.ravel()

    print(Density.max())

    p.update_scalars(Density, mesh=depth_centers_grid) 
    
    time_outputstep = trim.time.isel(time=outputstep).values
    timeActor = p.add_text(f"Density at {easyTimeFormat(time_outputstep)}", position='upper_left')

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(outline)
p.add_mesh(bottom_surface, cmap=deep, opacity=0.3)
p.add_mesh(depth_centers_grid, scalars='Density', clim=density_clim, opacity=opacity_trick, scalar_bar_args=sargs, **plotting_options)
initTimeActor = p.add_text(f"Density at {easyTimeFormat(time_outputstep)}", position='upper_left')
# p.add_slider_widget(densityCallback, time_lims, title="Outputstep", value=initial_timestep, event_type='end')
p.enable_depth_peeling()
p.show_grid()
p.set_scale(zscale=25)
panel = p.show()

# Interactive silt concentrations

In [ ]:
depth_centers_grid['Silt_concentration'] = concentration_silt.isel(time=initial_timestep).values.ravel()

In [ ]:
concentration_silt.isel(time=-10).max()

In [ ]:
concentration_silt.max()

In [ ]:
def siltCallback(val):
    p.remove_actor(initTimeActor)
    outputstep = int(val)
    Silt_concentration = concentration_silt.isel(time=outputstep).values.ravel()
    
    print(Silt_concentration.max())

    p.update_scalars(Silt_concentration, mesh=depth_centers_grid) 
    
    time_outputstep = trim.time.isel(time=outputstep).values
    timeActor = p.add_text(f"Silt conc at {easyTimeFormat(time_outputstep)}", position='upper_left')

In [ ]:
silt_clim = (0.1, 5)
silt_opacity_trick = [0] * 6 + [0.9] * 15 # should 'map' to steps of clim 

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(outline)
p.add_mesh(bottom_surface, cmap=deep, opacity=0.3)
p.add_mesh(depth_centers_grid, scalars='Silt_concentration', clim=silt_clim, opacity=silt_opacity_trick, scalar_bar_args=sargs, **plotting_options)
initTimeActor = p.add_text(f"Silt concentration at {easyTimeFormat(time_outputstep)}", position='upper_left')
p.add_slider_widget(siltCallback, time_lims, title="Outputstep", value=initial_timestep, event_type='end')
p.enable_depth_peeling()
p.show_grid()
p.set_scale(zscale=25)
panel = p.show()